- x86 is not clea
- Mips is clean
- "it is a simulator it must be broken" is wrong mindset, treat it like hardware
- Write Test Scripts
- Go Through the notes on C and compilation
- Get the book perhaps?
- Cookbook for 2 3
$$\text{asfasdklfj sldkfj asdflj }$$
- aklsdjfa

Concurrency is not parallelism(for the purpose of speed), concurrency is for functionality reasons. 
Concurrency they may or may not be running at the same time but assume that they are.

threads are an os abstraction: java uses ptreads
thread: program counter, each thread has its own stack(local variables live there), one global collection of global variables 
2:14
2:18
async vs sync
*null
malloc is void star
floating equal integer
4 = 4.0 representation will be different
coercesion int -> floating
dealing with pointers you use casts not coercions

determine seq. srand(unsigned int);. convetion: of 0. make it into the clock?
to divide double by int its a coercion
threads can only start at function boundary in C

void * is a pointer to anything

packaging the arguments is called martialling  / unpmartially
fflush insure that a print statement is set as immediately as possible, yes use fflush
context switch is a switch between threads

there is a disadvanteage of using one thread because of 140.      

what are the repercussions of not fflushing?
out_paramater is threadId
NULL = Default scheduling
pthread_create(out parameter thread id, schedule, function, argument)
you wait to join the threads
throw asserts in code

syncronishation : joining all threads together

---

-Multiprogramming : more than one thread at a time
- Basic problem of concurrency involves resources
    - Hardware: single CPU, single Dram, single I/O devices
    - Multiprogramming APIL users think they have exclusive access to shared resources
- OS has to coordinate all activity
    - Multiples users, I/O interrupts
- Multiplex in Time is the way to achieve this
    - each "cpu" needs to keep track of PC(Program Counter), SP(Stack Pointer) and registers in some state
    - what triggers switch the cpu's?
        - timer, voluntary yield where cpu will yield
        - I/o, windows 95 would crash because of this
- Using Hardware SMT: Simutanesoul multhi threading/ hyperthreading
    - higher utilization of processor resources and cycles
- Process: Operating sytem abstraction to represent what is needed to run a single program
    - A signle, sequential stream of execution in its own address space
    - **Sequential Program Execution Stream **
        - Code executed as a single sequential stream of execution
        - Includes state of CPU Registers
    - ** Protected Resoureces** 
        - Main Memory state (contents of address space)
        - I/o state (file descriptors)
    - PCB: Process Control Block
        - process state, number, counter, registers, memory limits, list of open files
    - Context Switch is switching contexts from process 0 to process 1.
- Create a process
    - Must construct a new PCB
    - Must set up a new page tables for address space
        - expensive
    - Copy I/O state
- Program ? Process ask in class
- Shared Memory communication just make an overlap of address spaces for shared 
- Modern Lightweight process has more than one thread
    - has one address space
    - you can have multiple threads executing in one process
    - every thread has the same memory space, global variables, heap
        - I/O sate, file system, network connections
    - every thread will have its own execution stack, registers but will share code, data, files kept in TCB **Thread control Block**
    
    
        

<img  src="Screen Shot 2016-01-09 at 9.53.05 PM.png"/>

``` c
int I_array[10];
int *J_array;

J_array = (int *)malloc(10 * sizeof(int));

printf("size of I_array: %d\n",sizeof(I_array));
printf("size of J_array: %d\n",sizeof(J_array));
```
    will print two different values even though I_array ad J_array are otherwise equivalent. Can you guess what the output is? Turns out that the sizeof() operator reports the byte size that the compiler sees in the code. Thus, sizeof(I_array) is 40 but sizeof(J_array) is 8 since on a 64-bit x86 (like the ones we use) all pointer values require 8 bytes of storage.

``` c
struct return_value
{
	int i1;
	int i2;
	double d1;
};

struct return_value *MyFunction()
{
	struct return_value *r;
	r = (struct return_value *)malloc(sizeof(struct return_value));
	r->i1 = 10;
	r->i2 = 12;
	r->d1 = 3.1415;

	return(r);
}
```
    where the fields i1, i2, and d1 are the two integers and the double. Notice that in this example, I have to use malloc() to allocate the memory for return value structure. The caller would need to free this memory when it was done with the three values -- every caller. If you forget one there is a memory leak.
    
``` c
void MyFunction(int *i1, int *i2, double *d1)
{
	*i1 = 10;
	*i2 = 12;
	*d1 = 3.1415;

	return;
}
```
    Here, the void type says that there is no return value. The function uses its parameters as pointers to the data types it produces and the * operator to store the values in those memory locations. These are called out parameters since they let the function push value out of its scope.
---
``` c
void NewFunction(int *i1, int **p1)
{
	int *j;

	*i1 = 10;

	j = (int *)malloc(sizeof(int));
	*j = 12;
	*p1 = j;

	return;
}

int main(int argc, char **argv)
{
	int I;
	int *J;

	NewFunction(&I,&J);

	printf("I: %d, J: %p, *J: %d\n",I,J,*J);
	free(J);

	return(0);
}
```
    It works this way. The function NewFunction() has two out parameters: one for an integer and another for a pointer to an integer. The caller passes the address of an integer &I and the address of a pointer to an integer &J as the arguments. The function mallocs the memory and passes a pointer to that memory through the second out parameter which main must then free.

---
``` c
int main(int argc, char **argv)
{
        int i;
	int next;
	char *p;
	char *a;
	char arg1[4096];
	int found = 0;
	int len;

	for(i=1; i < argc; i++) {
		p = argv[i];
		
		if(strncmp(p,"arg1:",strlen("arg1:")) == 0) {
		
			next = i + 1;
			if(next >= argc) {
				printf("arg1 value missing\n");
				exit(1);
			}
			a = argv[next];
			
			strncpy(arg1,a,sizeof(arg1));
			
			len = strlen(a);
			if(len >= sizeof(arg1)) {
				len = sizeof(arg1) - 1;
			}
			arg1[len] = 0;
			found = 1;
		}
	}

	if(found == 1) {
		printf("arg1 found with value %s\n",
				arg1);
	} else {
		printf("no arg1 value found\n");
	}

	return(0);
}
```			
    First off, what does the program do? It looks through the argument list passed to it for the string "arg1:" and then extracts into an array on the stack the next string. Thus, if your program's api was

    ./string3 arg1: first-argument

    it parses the argument list looking for the value immediately following the keyword "arg1:".
Run it a few times:

    ./string3
    no arg1 value found

    ./string3 arg1:
    arg1 value missing

    ./string3 arg1: dog!
    arg1 found with value dog!

    ./string3 arg1: happy dog!
    arg1 found with value happy

    ./string3 arg1: happy-dog!
    arg1 found with value happy-dog!

    ./string3 arg1 dog!
    no arg1 value found

    ./string3 Arg1: dog!
    no arg1 value found
